# Step 1 (Enviroment Setup)


---


## Importing the Datasets
wikitext (*full* ) - 859955 docs

wikitext (*small* ) - 10000 docs

In [1]:
!wget -O wikitext-filtered-full.zip "https://www.dropbox.com/scl/fi/ibd4cmixckghx6hhb361c/wikitext-filtered-full.zip?rlkey=q71cebf0k5fvvwhmcntoswzhq&dl=1"
!wget -O wikitext-filtered-10k.zip "https://www.dropbox.com/scl/fi/ek174r3sg7qjx0aa9atop/wikitext-filtered-10k.zip?rlkey=zy6jqxv6qsc16lr9qm3ki9uhf&dl=1"

--2025-10-09 20:07:43--  https://www.dropbox.com/scl/fi/ibd4cmixckghx6hhb361c/wikitext-filtered-full.zip?rlkey=q71cebf0k5fvvwhmcntoswzhq&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.70.18, 2620:100:6057:18::a27d:d12
Connecting to www.dropbox.com (www.dropbox.com)|162.125.70.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uccff996ba72a552ce0d939aa523.dl-eu.dropboxusercontent.com/cd/0/inline/Cy7aPsq_UKgH1asNm1c4qbSLSVf2FTGyLqDSEnoH6r44ymsGIrxIP5scDY_7C6KxGNL-psklCEUNODEH34FCBKD5NlUJfc-o2DAIFwV3OFf0tc57V8Gfu-IKgtFzvqaxsZ9lwrDeGkYSh5b0Gg785pBq/file?dl=1# [following]
--2025-10-09 20:07:44--  https://uccff996ba72a552ce0d939aa523.dl-eu.dropboxusercontent.com/cd/0/inline/Cy7aPsq_UKgH1asNm1c4qbSLSVf2FTGyLqDSEnoH6r44ymsGIrxIP5scDY_7C6KxGNL-psklCEUNODEH34FCBKD5NlUJfc-o2DAIFwV3OFf0tc57V8Gfu-IKgtFzvqaxsZ9lwrDeGkYSh5b0Gg785pBq/file?dl=1
Resolving uccff996ba72a552ce0d939aa523.dl-eu.dropboxusercontent.com (uccff996ba72a552ce0d939aa523.dl-eu.drop

In [2]:
!unzip wikitext-filtered-full.zip
!unzip wikitext-filtered-10k.zip

Archive:  wikitext-filtered-full.zip
   creating: wikitext-filtered-full/
  inflating: wikitext-filtered-full/dataset_info.json  
  inflating: wikitext-filtered-full/state.json  
  inflating: wikitext-filtered-full/data-00000-of-00001.arrow  
Archive:  wikitext-filtered-10k.zip
   creating: wikitext-filtered-10k/
  inflating: wikitext-filtered-10k/dataset_info.json  
  inflating: wikitext-filtered-10k/state.json  
  inflating: wikitext-filtered-10k/data-00000-of-00001.arrow  


In [3]:
# datasets package provides dataset tools from hugginface
!pip install datasets
import datasets

In [4]:
from datasets import load_dataset, Dataset

def load_dataset():
  wikitext_small = "wikitext-filtered-10k"
  wikitext_large = "wikitext-filtered-full"

  dataset_small = Dataset.load_from_disk(wikitext_small)
  dataset_large = Dataset.load_from_disk(wikitext_large)
  print("wikitext_small: {} docs, wikitext_large: {} docs".format(len(dataset_small), len(dataset_large)))
  return dataset_small, dataset_large

wikitext_small, wikitext_large = load_dataset()

wikitext_small: 10000 docs, wikitext_large: 859955 docs


## Understanding the Dataset
Summary statistics

In [5]:
wt = wikitext_small
#wt = wikitext_large

print('# TYPE OF THE DATASET:', '\n', type(wt))
print(wt, '\n')
print('# ENTRIES LOOK LIKE:')
print(wt.features, '\n', wt[0], '\n', wt[1], '\n')

print('# DATASET STATISTICS:')
print('No. of docs:', len(wt))
lengths = [len(doc['text'].split()) for doc in wt]
print('Mean doc length:', sum(lengths)/len(lengths), 'words')

# TYPE OF THE DATASET: 
 <class 'datasets.arrow_dataset.Dataset'>
Dataset({
    features: ['text'],
    num_rows: 10000
}) 

# ENTRIES LOOK LIKE:
{'text': Value('string')} 
 {'text': 'Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " .'} 
 {'text': "The game began development in 2010 , carrying over a large portion of the work done on Valkyria Ch

# Step 2 (Train Baselines)

---
Installing dependancies
- gensim - word2vec models
- nltk (natural language tool kit) - stopwords removal

In [1]:
!pip install gensim nltk
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess_dataset(dataset):
    text_col = 'text' if 'text' in dataset.column_names else dataset.column_names[0]
    tokenized = []

    for i in range(len(dataset)):
        text = dataset[i][text_col]
        if not isinstance(text, str):
            continue
        tokens = [t.lower() for t in text.split() if t.isalpha() and t.lower() not in stop_words]
        if tokens:
            tokenized.append(tokens)

    return tokenized

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
